In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc

In [2]:
df_md = pd.read_csv('for_cloud.csv')
df_stockwatch = pd.read_csv('merged_data_stockwatch.csv')
df_reut = pd.read_csv('own_data_test_sentiments.csv')

tickers = pd.read_csv('../constituents_csv.csv')
tickers = tickers.rename(index = str, columns = {'Symbol': 'ticker'})

C:\Users\zhav1k\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_md['shifted_close'] = df_md.groupby('ticker')['Close'].shift(-1)
df_md['tgt'] = (df_md.shifted_close - df_md.Close).div(df_md.shifted_close) * 100
df_md['target'] = 0
df_md.loc[(df_md['tgt'] >= 0), 'target'] = 1
df_md.loc[(df_md['tgt'] < 0), 'target'] = 0
df_md.drop([ 'Sector', 'shifted_close', 'tgt'], axis = 1 , inplace = True)

In [4]:
df_stockwatch['Date'] = pd.to_datetime(df_stockwatch['Date'])
df_stockwatch['Date'] = df_stockwatch['Date'].dt.date

df_md['Date'] = pd.to_datetime(df_md['Date'])
df_md['Date'] = df_md['Date'].dt.date

df_reut['Date'] = pd.to_datetime(df_reut['Date'])
df_reut['Date'] = df_reut['Date'].dt.date

df_reut = pd.merge(df_reut, tickers, how = 'left', on = 'Name')

In [5]:
df_reut.shape

(887405, 6)

In [7]:
df_stockwatch.shape

(279876, 3)

In [41]:
df_reut.drop(['Sname', 'Name', 'Sector'], axis = 1, inplace = True)
fcols = ['Date', 'ticker', 'title']
df_reut = df_reut[fcols]

In [42]:
df_news = pd.concat([df_reut, df_stockwatch], axis = 0)

In [43]:
df_gen = pd.merge(df_md,df_news,on=['ticker', 'Date'], how = 'left')

In [44]:
df_gen1 = df_gen[df_gen['title'].notnull()]

df_gen1 = pd.DataFrame(df_gen1.groupby(['ticker', 'Date'])['title'].agg(lambda col: ' '.join(col)))
df_gen1 = df_gen1.reset_index()

In [45]:
to_join = pd.DataFrame(df_gen.groupby(['ticker', 'Date'])['target'].max())
to_join = to_join.reset_index()

In [46]:
merged = pd.merge(df_gen1, to_join, on = ['ticker', 'Date'], how = 'left')

In [51]:
merged = merged.reset_index()
merged.drop(['ticker', 'Date', 'target'], axis = 1, inplace =True)

In [52]:
merged.to_csv('data_sentiments/test.tsv', sep='\t', index=False, header=True)